In [33]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 100)

import qgrid

In [34]:
file = '/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/AAP.csv'

In [35]:
df = pd.read_csv(file)

In [36]:
df.head(2)

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,outstandingShares,Revenue,Gross Profit,R&D Expenses,Op Expenses,Op Income,Net Income,Cash,Receivables,Inventory,Cur Assets,Tax Assets,LT Assets,Int Assets,Total Assets,Payables,Deferred Revenue Current,Deferred Revenue LongTerm,Cur Liab,LT Debt,LT Liab,Total Liab,SH Equity,cash_flow Operations,cash_flow Investing,cash_flow Financing,CAPEX,Fcash_flow,Dividends Paid,Gross Profit Margin,Op Margin,Int Coverage,Net Profit Margin,Dividend Yield,Current Ratio,Operating Cycle,Days of AP Outstanding,Cash Conversion Cycle,ROA,ROE,ROCE,PE,PS,PB,Price To FCF,PEG,EPS,Ticker
0,2022-06-25,2021,Q4,"14,937.81",1.08,0.74,38.49,1.31,62.84,"2,396.97","1,072.12",0.00,959.65,112.46,81.67,138.73,957.80,"4,778.85","6,057.78",0.00,"5,944.70",642.12,"12,002.48","3,942.39",0.00,0.00,"5,052.57","3,517.70","4,040.23","9,092.80","2,909.68",187.33,-98.43,-95.37,-98.66,88.67,0.00,0.45,0.05,11.59,0.03,NaN,1.21,345.89,266.43,79.46,0.01,0.03,0.02,45.73,6.23,4.78,168.47,-21.03,1.31,AAP
1,2022-03-26,2021,Q3,"13,460.18",1.00,0.73,41.70,2.70,63.35,"2,621.23","1,182.45",0.00,953.26,229.20,169.81,601.43,782.78,"4,659.02","6,275.48",0.00,"5,918.73",651.22,"12,194.21","3,922.01",0.00,0.00,"5,180.31","3,371.97","3,885.61","9,065.92","3,128.29",148.78,-61.16,-292.30,-61.41,87.37,-62.95,0.45,0.09,26.69,0.06,0.00,1.24,310.38,234.63,75.76,0.01,0.05,0.03,19.82,5.14,4.21,154.06,-8.01,2.70,AAP


## Quality

In [37]:
def asset_growth(ticker):
    """Asset growth is defined as the YoY % change in quarterly total assets.
    Higher is worse, so multiply by -1"""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['Asset_growth'] = (df.groupby('Quarter')['Total Assets'].pct_change()[1:]*-1)*100
    return df[['Calendar_Year', 'Quarter','Total Assets','Asset_growth']]

In [38]:
asset_growth('AAPL').tail(1)

,Calendar_Year,Quarter,Total Assets,Asset_growth
0,2022,Q3,"336,309.00",-1.96


### Account clarity

`Account Clarity`: 
+ 12 Quarter Avg of:
+ current OP assets - OP liabilities
+ less 4 quarter ago OP assets - Op liabilities
+ scaled by average assets over the same period

where:
+ Operating Assets = Total Assets - cash
+ Operating Liability = Total Assets - debt - minority interests - common & pref. stock 

In [39]:
df.head(1)

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,outstandingShares,Revenue,Gross Profit,R&D Expenses,Op Expenses,Op Income,Net Income,Cash,Receivables,Inventory,Cur Assets,Tax Assets,LT Assets,Int Assets,Total Assets,Payables,Deferred Revenue Current,Deferred Revenue LongTerm,Cur Liab,LT Debt,LT Liab,Total Liab,SH Equity,cash_flow Operations,cash_flow Investing,cash_flow Financing,CAPEX,Fcash_flow,Dividends Paid,Gross Profit Margin,Op Margin,Int Coverage,Net Profit Margin,Dividend Yield,Current Ratio,Operating Cycle,Days of AP Outstanding,Cash Conversion Cycle,ROA,ROE,ROCE,PE,PS,PB,Price To FCF,PEG,EPS,Ticker
0,2022-06-25,2021,Q4,"14,937.81",1.08,0.74,38.49,1.31,62.84,"2,396.97","1,072.12",0.00,959.65,112.46,81.67,138.73,957.80,"4,778.85","6,057.78",0.00,"5,944.70",642.12,"12,002.48","3,942.39",0.00,0.00,"5,052.57","3,517.70","4,040.23","9,092.80","2,909.68",187.33,-98.43,-95.37,-98.66,88.67,0.00,0.45,0.05,11.59,0.03,NaN,1.21,345.89,266.43,79.46,0.01,0.03,0.02,45.73,6.23,4.78,168.47,-21.03,1.31,AAP


In [50]:
def operating_assets(ticker, window):
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['current assets'] = (df['Total Assets'].rolling(window).sum() - df['Cash'].rolling(window).sum())
    return df['current assets']

def operating_liability(ticker, window):
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['current liabilities'] = (df['Total Assets'].rolling(window).sum() - df['Cash'].rolling(window).sum())
    return df

def operating_assets_12q_avg(ticker):
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['operating_assets_12q_avg'] = operating_assets(ticker, 12)
    return df[['Calendar_Year', 'Quarter', 'Mkt Cap', 'Ticker','operating_assets_12q_avg']]

def account_clarity(ticker):
    """12 Quarter Avg of:
    current OP assets - OP liabilities
    less 4 quarter ago OP assets - Op liabilities
    scaled by average assets over the same period 
     -------------------------------------------
     
     Operating Assets = Total Assets - cash
     Operating Liability = Total Assets - debt - minority interests - common & pref. stock 
     """
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    avg_assets_12q = df['Total Assets'].rolling(12).mean()
    assets_4q  = df['Total Assets'].shift(4) - df['']


In [51]:
operating_assets_12q_avg('AAPL').tail()

,Calendar_Year,Quarter,Mkt Cap,Ticker,operating_assets_12q_avg
4,2021,Q3,"2,213,535.58",AAPL,"3,597,583.00"
3,2021,Q4,"2,422,287.79",AAPL,"3,573,833.00"
2,2022,Q1,"2,955,919.62",AAPL,"3,588,957.00"
1,2022,Q2,"2,863,962.04",AAPL,"3,607,511.00"
0,2022,Q3,"2,289,642.85",AAPL,"3,644,609.00"
